In [1]:
import pandas as pd
import os
import numpy as np
#np.set_printoptions(precision=4)
import catboost
from catboost import *

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import precision_score, recall_score, roc_auc_score, f1_score, classification_report

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
Data_Matching = pd.read_excel('19.09.23 список образцов Моча.xlsx', header=2).loc[:,['Порядковый номер', 'TOTAL']]
Dataset = pd.read_csv('Spectra_Dataset_Chms_1_99.csv')
Dataset_PCA = pd.read_csv('PCA_Dataset_Chms_1_99.csv')

In [3]:
Indecies_to_split = Data_Matching.dropna()

In [4]:
Indecies_to_split['TOTAL_bin'] = Indecies_to_split.TOTAL.apply(lambda x: 0 if x==0 else 1)

def mapping_dict(x):
    if x==0:
        return 0
    if x==1:
        return 1
    if (x>=2)&(x<=5):
        return 2
    if (x>=6):
        return 3
    
    
Indecies_to_split['TOTAL_rank'] = Indecies_to_split.TOTAL.apply(mapping_dict)

In [5]:
Exam_Indicies = Indecies_to_split.iloc[-10:,:]
Indecies_to_split = Indecies_to_split.iloc[:-10,:]

In [7]:
Chms_bin_train, Chms_bin_test, y_bin_train, y_bin_test = train_test_split(Indecies_to_split['Порядковый номер'],
                                                          Indecies_to_split.TOTAL_bin,
                                                         test_size=0.2, stratify=Indecies_to_split.TOTAL_bin,
                                                         random_state=42)

Chms_rank_train, Chms_rank_test, y_rank_train, y_rank_test = train_test_split(Indecies_to_split['Порядковый номер'],
                                                          Indecies_to_split.TOTAL_rank,
                                                         test_size=0.2, stratify=Indecies_to_split.TOTAL_rank,
                                                         random_state=42)

In [8]:
Train_y = Indecies_to_split[list(map(lambda x: x in Chms_bin_train.values, Indecies_to_split['Порядковый номер']))]
Test_y = Indecies_to_split[list(map(lambda x: x in Chms_bin_test.values, Indecies_to_split['Порядковый номер']))]



In [16]:
Dataset.columns

Index(['Dataset', 'Description', 'Ch_0_0', 'Ch_0_1', 'Ch_0_2', 'Ch_0_3',
       'Ch_0_4', 'Ch_0_5', 'Ch_0_6', 'Ch_0_7', 'Ch_0_8', 'Ch_0_9', 'Ch_0_10',
       'Ch_0_11', 'Ch_0_12', 'Ch_0_13', 'Ch_0_14', 'Ch_0_15', 'Ch_0_16',
       'Ch_0_17', 'Ch_1_0', 'Ch_1_1', 'Ch_1_2', 'Ch_1_3', 'Ch_1_4', 'Ch_1_5',
       'Ch_1_6', 'Ch_1_7', 'Ch_1_8', 'Ch_1_9', 'Ch_1_10', 'Ch_1_11', 'Ch_1_12',
       'Ch_1_13', 'Ch_1_14', 'Ch_1_15', 'Ch_1_16', 'Ch_1_17',
       'Порядковый номер', 'Пол', 'Возраст', 'Цвет', 'TOTAL', 'Плотность.1',
       'pH.1', 'Белок.1', 'Билирубин.1', 'Глюкоза.1', 'Кетоны.1',
       'Лейкоциты.1', 'Нитриты.1', 'Уробилиноген.1', 'Кровь.1',
       'Клетки плоского эпителия.1', 'Цилиндры гиалиновые.1', 'Бактерии.1',
       'Кристаллы.1', 'Дрожжи.1', 'Малые клетки.1',
       'Цилиндры патологические.1', 'Слизь.1', 'Сперматозоиды.1'],
      dtype='object')

In [17]:
NeedCols_from_PCA = ['PC_0', 'PC_1', 'PC_2', 'PC_3', 'PC_4', 'PC_5', 'Dataset', 'Пол', 'Возраст', 'TOTAL',
                    'pH.1', 'Белок.1', 'Билирубин.1', 'Глюкоза.1', 'Кетоны.1',
                    'Лейкоциты.1', 'Нитриты.1', 'Уробилиноген.1', 'Кровь.1',
                    'Клетки плоского эпителия.1', 'Цилиндры гиалиновые.1', 'Бактерии.1',
                    'Кристаллы.1', 'Дрожжи.1', 'Малые клетки.1',
                    'Цилиндры патологические.1', 'Слизь.1', 'Сперматозоиды.1']


NeedCols_from_Spectra = list(Dataset.filter(regex='^Ch').columns)+['Dataset', 'Пол', 'Возраст', 'TOTAL','Плотность.1',
                                                                   'pH.1', 'Белок.1', 'Билирубин.1', 'Глюкоза.1', 'Кетоны.1',
                                                                   'Лейкоциты.1', 'Нитриты.1', 'Уробилиноген.1', 'Кровь.1',
                                                                   'Клетки плоского эпителия.1', 'Цилиндры гиалиновые.1', 'Бактерии.1',
                                                                   'Кристаллы.1', 'Дрожжи.1', 'Малые клетки.1',
                                                                   'Цилиндры патологические.1', 'Слизь.1', 'Сперматозоиды.1']


TRAIN = Dataset[list(map(lambda x: x in Chms_bin_train.values, Dataset.Dataset))].loc[:,NeedCols_from_Spectra]


TEST = Dataset[list(map(lambda x: x in Chms_bin_test.values, Dataset.Dataset))].loc[:,NeedCols_from_Spectra]

EXAM = Dataset[list(map(lambda x: x in Exam_Indicies['Порядковый номер'].values, Dataset.Dataset))].loc[:,NeedCols_from_Spectra]

In [18]:
TRAIN['TOTAL_bin'] = TRAIN.TOTAL.apply(lambda x: 0 if x==0 else 1)
TRAIN['TOTAL_rank'] = TRAIN.TOTAL.apply(mapping_dict)
train_cols = TRAIN.columns
TRAIN = pd.DataFrame(TRAIN.values, columns=train_cols)

In [19]:
TEST['TOTAL_bin'] = TEST.TOTAL.apply(lambda x: 0 if x==0 else 1)
TEST['TOTAL_rank'] = TEST.TOTAL.apply(mapping_dict)
test_cols = TEST.columns
TEST = pd.DataFrame(TEST.values, columns=test_cols)

In [20]:
EXAM['TOTAL_bin'] = EXAM.TOTAL.apply(lambda x: 0 if x==0 else 1)
EXAM['TOTAL_rank'] = EXAM.TOTAL.apply(mapping_dict)
EXAM_cols = EXAM.columns
EXAM = pd.DataFrame(EXAM.values, columns=EXAM_cols)

In [21]:
TRAIN_X = TRAIN.loc[:, NeedCols_from_Spectra[:-4]+['Пол','Возраст']]
TRAIN_y = TRAIN['TOTAL_bin']

TEST_X = TEST.loc[:, NeedCols_from_Spectra[:-4]+['Пол','Возраст']]
TEST_y = TEST['TOTAL_bin']

EXAM_X = EXAM.loc[:, NeedCols_from_Spectra[:-4]+['Пол','Возраст']]
EXAM_y = EXAM['TOTAL_bin']

In [35]:
TRAIN.columns[list(map(lambda x: x.endswith('.1'), TRAIN.columns))]

Index(['Плотность.1', 'pH.1', 'Белок.1', 'Билирубин.1', 'Глюкоза.1',
       'Кетоны.1', 'Лейкоциты.1', 'Нитриты.1', 'Уробилиноген.1', 'Кровь.1',
       'Клетки плоского эпителия.1', 'Цилиндры гиалиновые.1', 'Бактерии.1',
       'Кристаллы.1', 'Дрожжи.1', 'Малые клетки.1',
       'Цилиндры патологические.1', 'Слизь.1', 'Сперматозоиды.1'],
      dtype='object')

In [14]:
TRAIN_X

,Ch_0_0,Ch_0_1,Ch_0_2,Ch_0_3,Ch_0_4,Ch_0_5,Ch_0_6,Ch_0_7,Ch_0_8,Ch_0_9,...,Ch_1_10,Ch_1_11,Ch_1_12,Ch_1_13,Ch_1_14,Ch_1_15,Ch_1_16,Ch_1_17,Пол,Возраст
0,0.00954787,0.00647428,0.0496854,0.202149,0.123593,0.387389,0.0411951,0.104526,0.142864,0.0218039,...,0.074371,0.0753712,0.0411478,0.0606071,0.00931409,0.0189395,0.00818947,0.0127479,m,24
1,0.00894421,0.00674,0.0494082,0.213435,0.125622,0.417335,0.0405,0.107062,0.146123,0.0230812,...,0.0740401,0.0710081,0.0403004,0.0619662,0.0097886,0.0196523,0.00847976,0.0137004,m,24
2,0.00739949,0.00526416,0.0445294,0.188678,0.115589,0.379931,0.0363396,0.0986831,0.138609,0.0214772,...,0.0702422,0.0690503,0.0363412,0.0596908,0.00883181,0.0194317,0.00771385,0.0109389,m,24
3,0.0082852,0.00591103,0.0482445,0.200886,0.123995,0.399413,0.0384849,0.102645,0.144699,0.0221994,...,0.0727812,0.0698202,0.0387709,0.060423,0.00944023,0.0197635,0.00813398,0.0136645,m,24
4,0.00744066,0.00525494,0.0440432,0.185445,0.113481,0.372787,0.0363931,0.0969895,0.136853,0.021068,...,0.0704908,0.069071,0.0363981,0.0522387,0.00872581,0.0196841,0.00756689,0.0109166,m,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
743,0.0110538,0.00678732,0.0492935,0.218457,0.11693,0.511628,0.0439023,0.0951778,0.122898,0.0202864,...,0.0707336,0.0734867,0.0404734,0.0622011,0.0080675,0.0196268,0.00732614,0.0126538,f,70
744,0.0112107,0.00659133,0.0480208,0.212992,0.112842,0.504056,0.0441472,0.0946414,0.120003,0.0200138,...,0.0703694,0.0767234,0.0405111,0.059318,0.00785889,0.0212808,0.00731171,0.0116941,f,70
745,0.0110055,0.0067577,0.048922,0.217035,0.116256,0.508254,0.0437107,0.0946014,0.122084,0.0201979,...,0.0708577,0.0734546,0.0406666,0.0621145,0.00792474,0.0195411,0.00729417,0.0125986,f,70
746,0.0107156,0.00648817,0.0478283,0.214044,0.113344,0.498913,0.0429393,0.0940663,0.120725,0.0200203,...,0.0702931,0.073739,0.0398387,0.0613676,0.00791322,0.0195808,0.00717864,0.0129374,f,70


,Ch_0_0,Ch_0_1,Ch_0_2,Ch_0_3,Ch_0_4,Ch_0_5,Ch_0_6,Ch_0_7,Ch_0_8,Ch_0_9,...,Цилиндры гиалиновые.1,Бактерии.1,Кристаллы.1,Дрожжи.1,Малые клетки.1,Цилиндры патологические.1,Слизь.1,Сперматозоиды.1,TOTAL_bin,TOTAL_rank
0,0.00989977,0.00273175,0.0213867,0.0115699,0.0356104,0.0437034,0.0167045,0.0198026,0.0681787,0.00695995,...,1,0,1,0,1,1,1,0,1,3
1,0.00785071,0.00211092,0.0176591,0.0101033,0.0304199,0.0399101,0.0141234,0.0182522,0.0606766,0.00695014,...,1,0,1,0,1,1,1,0,1,3
2,0.0087472,0.00271302,0.0193809,0.0115997,0.032486,0.0438157,0.0151824,0.0201907,0.0630918,0.00718609,...,1,0,1,0,1,1,1,0,1,3
3,0.00918619,0.00291735,0.0201434,0.0115154,0.0334346,0.0431013,0.0155402,0.0201474,0.0642549,0.00716532,...,1,0,1,0,1,1,1,0,1,3
4,0.00830224,0.00240865,0.0185308,0.0107726,0.0304911,0.0407311,0.0149099,0.0193123,0.0618214,0.00707752,...,1,0,1,0,1,1,1,0,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,0.00736137,0.00380651,0.0301615,0.0890291,0.074441,0.190117,0.0255419,0.0636343,0.112002,0.0154413,...,0,0,0,0,0,0,0,0,0,0
195,0.00722392,0.00387151,0.0303439,0.0900985,0.0748698,0.185629,0.0256124,0.0636728,0.112516,0.0152463,...,0,0,0,0,0,0,0,0,0,0
196,0.00748038,0.00394491,0.0306491,0.0919964,0.0759632,0.179245,0.0258761,0.0645058,0.113405,0.015623,...,0,0,0,0,0,0,0,0,0,0
197,0.00772399,0.00402856,0.0312366,0.0935293,0.0771294,0.184257,0.0257622,0.0659545,0.111896,0.0155134,...,0,0,0,0,0,0,0,0,0,0
